In [1]:
%%javascript
 
Jupyter.keyboard_manager.command_shortcuts.remove_shortcut('up');
Jupyter.keyboard_manager.command_shortcuts.remove_shortcut('down')


<IPython.core.display.Javascript object>

In [1]:
import pandas as pd
import calendar
from datetime import datetime
import _pickle as cPickle
import os
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
test_dataset = pd.read_csv("sample_test.csv")

In [3]:
test_dataset['sensor_id'] = test_dataset.sensor_id.astype("category")
#cleaning the start and end time
test_dataset['start_time'] = test_dataset['start_time'].astype("str")
test_dataset['start_time'] = test_dataset['start_time'].str.replace('T',' ')
test_dataset['start_time'] = test_dataset['start_time'].str.replace('Z','')

test_dataset['end_time'] = test_dataset['end_time'].str.replace('T',' ')
test_dataset['end_time'] = test_dataset['end_time'].str.replace('Z','')

test_dataset['start_time'] = test_dataset.start_time.str[1:-1]
test_dataset['end_time'] = test_dataset.end_time.str[1:-1]

### Rule based algorithm to derive shower duration 

In [4]:
from datetime import datetime
test_shower_activity_recognition = test_dataset[(test_dataset.sensor_id == 3802) & (test_dataset.is_original ==1) & 
                                         (test_dataset.value == 1)]



test_shower_activity_recognition['start_time'] = pd.to_datetime(test_shower_activity_recognition['start_time'], 
                                                           format='%Y-%m-%d %H:%M:%S.%f')
test_shower_activity_recognition['end_time'] = pd.to_datetime(test_shower_activity_recognition['end_time'], 
                                                              format='%Y-%m-%d %H:%M:%S.%f')

test_shower_activity_recognition['Duration_seconds'] = (test_shower_activity_recognition.end_time - 
                                                test_shower_activity_recognition.start_time).astype('timedelta64[s]')

test_shower_activity_recognition['minutes'] = test_shower_activity_recognition.end_time.sub(test_shower_activity_recognition.start_time).dt.total_seconds().div(60)
test_shower_activity_recognition['minutes'] = test_shower_activity_recognition.minutes.astype(int)

In [5]:
test_shower_activity_recognition.dtypes

id                           int64
sensor_id                 category
value                      float64
start_time          datetime64[ns]
end_time            datetime64[ns]
duration                   float64
is_original                  int64
Duration_seconds           float64
minutes                      int32
dtype: object

### Knowing the Duration between each subsequent Shower

In [6]:
test_non_shower_usage_duration = test_dataset[(test_dataset.sensor_id == 3802) & (test_dataset.is_original ==1) & 
                                         (test_dataset.value == 0)]


test_non_shower_usage_duration.drop("duration",inplace=True,axis=1)
test_non_shower_usage_duration['start_time'] = pd.to_datetime(test_non_shower_usage_duration['start_time'],
                                                              format='%Y-%m-%d %H:%M:%S.%f')
test_non_shower_usage_duration['end_time'] = pd.to_datetime(test_non_shower_usage_duration['end_time'],
                                                            format='%Y-%m-%d %H:%M:%S.%f')

test_non_shower_usage_duration['duration'] = (test_non_shower_usage_duration.end_time -
                                              test_non_shower_usage_duration.start_time).astype('timedelta64[s]')

test_non_shower_usage_duration['minutes'] = test_non_shower_usage_duration.end_time.sub(test_non_shower_usage_duration.start_time).dt.total_seconds().div(60)
test_non_shower_usage_duration['minutes'] = test_non_shower_usage_duration.minutes.astype(int)

In [7]:
test_non_shower_usage_duration.head()

,id,sensor_id,value,start_time,end_time,is_original,duration,minutes
2,4998417,3802,0.0,2017-06-01 10:36:28,2017-06-01 18:50:48,1,29660.0,494
4,5002934,3802,0.0,2017-06-01 18:54:47,2017-06-01 18:59:02,1,255.0,4
6,5003025,3802,0.0,2017-06-01 19:13:07,2017-06-02 04:00:00,1,31613.0,526
9,5014581,3802,0.0,2017-06-02 19:44:52,2017-06-03 04:00:00,1,29708.0,495
12,5025240,3802,0.0,2017-06-03 19:14:34,2017-06-04 04:00:00,1,31526.0,525


### Testing ML Model

In [8]:

cwd = os.getcwd()
test_dataset.drop("id",axis=1,inplace=True)
test_dataset['is_original'] = test_dataset.is_original.astype("category")

In [9]:
test_dataset['start_time'] = test_dataset.start_time.astype("str")
test_dataset["date"] = test_dataset.start_time.apply(lambda x : x.split()[0])
test_dataset["hour"] = test_dataset.start_time.apply(lambda x : x.split()[1].split(":")[0])
test_dataset["weekday"] = test_dataset.date.apply(lambda dateString : calendar.day_name[datetime.strptime(dateString,"%Y-%m-%d").weekday()])

In [10]:
test_dataset["sensor"] = test_dataset.sensor_id.map({3802: "In Shower", 3803 : "Temperature in Shower",
                                                 3804 : "Light Levels in Shower", 3805 :"Humidity in Shower",
                                                 3807 : "In Bathroom", 3809 : "Toilet Flushed, Master" })


In [11]:
def round_int(x):
    if x == float("inf") or x == float("-inf"):
        return float('nan')
    return int(round(x))

In [12]:
test_dataset['value_roundoff'] = test_dataset.value.apply(round_int)

In [13]:
test_dataset['duration_roundoff'] = test_dataset.duration.apply(round_int)

In [14]:
test_dataset["weekday_num"] = test_dataset.weekday.map({"Sunday" : 0, "Monday" : 1,
                                                 "Tuesday" : 2, "Wednesday" : 3,
                                                 "Thursday" : 4, "Friday" : 5,
                                                     "Saturday" : 6})


In [15]:
categoricalFeatureNames = ["sensor_id","is_original","hour","weekday_num"]
numericalFeatureNames = ["value_roundoff","duration_roundoff"]
dropFeatures = ['value',"start_time","end_time","duration","date","weekday"]

In [16]:
data_cleaned_test = pd.DataFrame()
for var in categoricalFeatureNames:
    data_cleaned_test[var] = test_dataset[var].astype("category")

for var in numericalFeatureNames:
    data_cleaned_test[var] = test_dataset[var].astype("int")


In [17]:
data_cleaned_test = data_cleaned_test[data_cleaned_test['duration_roundoff'] <= 900]

In [18]:
X_testing = data_cleaned_test.drop("duration_roundoff",axis=1)
y_testing = data_cleaned_test['duration_roundoff']

In [19]:
def rmsle(y, y_,convertExp=True):
    if convertExp:
        y = np.exp(y),
        y_ = np.exp(y_)
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

## Applying Random Forest

In [20]:
with open(cwd + "\\rf.cpickle", 'rb') as f:
    rf_model = cPickle.load(f)

In [21]:
from sklearn.ensemble import RandomForestRegressor #importing random forest regressor
preds = rf_model.predict(X= X_testing)

y_testing_log = np.log1p(y_testing)

print ("RMSLE Value For Random Forest: ",rmsle(np.exp(y_testing_log),np.exp(preds),False))

RMSLE Value For Random Forest:  0.051619906802925816


## Applying GBM

In [23]:
filename = 'gbm.sav'
gbm_model = cPickle.load(open(filename, 'rb'))

In [24]:
y_log_test = y_log_test = np.log1p(y_testing)
pred_y_test = gbm_model.predict(X= X_testing)
print ("RMSLE Value For Gradient Boost on Test Dataset: ",rmsle(np.exp(y_log_test),np.exp(pred_y_test),False))

RMSLE Value For Gradient Boost on Test Dataset:  0.059906899228998896
